# Day 1 - Trebuchet

The newly-improved calibration document consists of lines of text; each line originally contained a specific calibration value that the Elves now need to recover. On each line, the calibration value can be found by combining the first digit and the last digit (in that order) to form a single two-digit number.

For example:
```
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
```
In this example, the calibration values of these four lines are 12, 38, 15, and 77. Adding these together produces 142.

Consider your entire calibration document. What is the sum of all of the calibration values?

In [ ]:
# Day 1, challenge 1

def calsum(filename):
    calvals = []
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for char in caldoc:
            # if it's a digit store it
            if char.isdigit():
                #Just keep updating the second digit every time you see a new one
                second_digit = int(char)
                #Only update the first digit the first time we see one
                if first_digit < 0:
                    first_digit = int(char)
            #if there's a new line or end of file, store the digit for this line and reset
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                # reset the vars for the digits
                first_digit = -1
                second_digit = -1

        # handle the case if the last line doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum('AoC23-1CalibrationDoc.txt')

# Day 1 - Challenge 2

Your calculation isn't quite right. It looks like some of the digits are actually spelled out with letters: one, two, three, four, five, six, seven, eight, and nine also count as valid "digits".

Equipped with this new information, you now need to find the real first and last digit on each line. For example:
```
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
```
In this example, the calibration values are 29, 83, 13, 24, 42, 14, and 76. Adding these together produces 281.

What is the sum of all of the calibration values?


In [ ]:
# Day 1, challenge 2

def calsum2(filename):
    calvals = []
    numbers = ['one','two','three','four','five','six','seven','eight','nine']
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1
        doclength = len(caldoc)

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for i,char in enumerate(caldoc):
            if char.isdigit():
                second_digit = int(char)
                if first_digit < 0:
                    first_digit = int(char)
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                first_digit = -1
                second_digit = -1
            else:
                for n,number in enumerate(numbers):
                    if len(number) + i > doclength:
                        next
                    elif caldoc[i:i+len(number)] == number:
                        second_digit = n + 1
                        if first_digit < 0:
                            first_digit = n + 1
                            

        # handle the case if the last line somehow doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum2('AoC23-1CalibrationDoc.txt')

# Day 2 - Game 1

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

In [ ]:
# Day 2, challenge 1
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors and values basec on cube counts that we're testing against
# Returns: sum of game ids that are valid
def cubegame(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    
    # re for matching game id
    # Slightly brittle, assumes Game is capitalized
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching the colors in each grab.
    # Slightly brittle, assumes number comes first and nothing is capitalized
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None           

            # if the maximum number we saw across the grabs is not greater than the target, we know that target is possible
            if game_tally['red'] <= target['red'] and game_tally['blue'] <= target['blue'] and game_tally['green'] <= target['green']:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            game_tally = {'red':'0','blue':'0','green':'0'}
            
    return possible_sum

cubegame('AoC23-2Game.txt',noisy=False)

# Day 2, Game 2

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
    In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
    Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
    Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
    Game 4 required at least 14 red, 3 green, and 15 blue cubes.
    Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.

The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [ ]:
# Day 2, challenge 2
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors (red,green,blue) and values based on cube counts that we're testing against
# Returns: sum of game ids that are valid, sum of power for all games
def cubegame2(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will hold the sum of the powers
    power_sum = 0
    # list of colors for iteration purposes.
    # Compared to challenge one version, this would enable us to easily expand to more colors
    colors = ['red','green','blue']
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    # This is the difference between the target and the game_tally
    game_diff = {'red':0,'blue':0,'green':0}
    
    #re for matching game id
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching each grab
    # To do: will need to modify this if we add colors
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # Reset things. tally to 0 and power to 1 because multiplication
            power = 1
            game_tally = {'red':'0','blue':'0','green':'0'}
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None
            
            # Generalized a bit from above. Instead of specifying red, green blue, iterate through list of colors
            for color in colors:                
                # The game tally is by definition the minimum you'd need for the game so just multiply each color
                power = power * game_tally[color]
                # calculate a diff from the target for testing valid games in challenge 1
                game_diff[color] = target[color] - game_tally[color]

            # Add power to the running sum
            power_sum += power
            print(f"Power = {power}, Power Sum = {power_sum}") if noisy else None
                
            if game_diff['red'] >= 0 and game_diff['blue'] >= 0 and game_diff['green'] >= 0:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            
    return possible_sum, power_sum

cubegame2('AoC23-2Game.txt',noisy=False)